In [44]:
import collections
import string
import itertools
import re

In [45]:
Link = collections.namedtuple('Link', 'height left right')

In [46]:
def extract_pairs(net_string):
    return [[int(pi) for pi in p.split(', ')] for p in net_string[1:-1].split('), (')]

In [47]:
def read_net_string(net_string):
    return [Link(h, l, r) for h, (l, r) in enumerate(extract_pairs(net_string))]

In [48]:
def read_net(filename, rev=False):
    with open(filename) as f:
        pairs = [re.split('\D+', p.strip()) for p in f.readlines()]
        if rev:
            lrs = [(int(lr[1]), int(lr[2])) for lr in reversed(pairs)]
        else:
            lrs = [(int(lr[1]), int(lr[2])) for lr in pairs]
        return [Link(h, l, r) 
                for h, (l, r) in enumerate(lrs)]

In [15]:
small_net = read_net('04-small.txt')
small_net

[Link(height=0, left=2, right=5),
 Link(height=1, left=1, right=4),
 Link(height=2, left=0, right=3),
 Link(height=3, left=0, right=3),
 Link(height=4, left=0, right=5),
 Link(height=5, left=3, right=5),
 Link(height=6, left=0, right=2),
 Link(height=7, left=3, right=4),
 Link(height=8, left=2, right=4),
 Link(height=9, left=1, right=2),
 Link(height=10, left=0, right=4),
 Link(height=11, left=1, right=2),
 Link(height=12, left=2, right=4),
 Link(height=13, left=0, right=4),
 Link(height=14, left=1, right=4)]

In [16]:
net = read_net('04-lines.txt')
len(net)

10135

In [36]:
permnet = read_net('permutations.txt')
len(permnet)

23

In [41]:
rpermnet = read_net('permutations.txt', rev=True)
len(rpermnet)

23

In [18]:
def show_net(links, pair_sep=', '):
    return pair_sep.join('({}, {})'.format(l.left, l.right) for l in sorted(links))

In [19]:
def link_ends(link):
    return set((link.left, link.right))

In [20]:
def follow(initial_line, links):
    line = initial_line
    heights = sorted(set(l.height for l in links))
    for h in heights:
        for l in [l for l in links if l.height == h]:
            if line in link_ends(l):
                line = [e for e in link_ends(l) if e != line][0]
#                 print(l, line)
    return line

In [21]:
def pack(net):
    packed_links = []
    line_heights = collections.defaultdict(lambda: -1)
    for link in sorted(net):
        link_height = max(line_heights[link.left], line_heights[link.right]) + 1
        line_heights[link.left] = link_height
        line_heights[link.right] = link_height
        packed_links += [Link(link_height, link.left, link.right)]
    return sorted(packed_links)

In [22]:
max(l.height for l in small_net)

14

In [23]:
max(l.height for l in pack(small_net))

10

In [24]:
max(l.height for l in net)

10134

In [25]:
pnet = pack(net)

In [26]:
max(l.height for l in pnet)

2286

In [27]:
def height_groups(net):
    return {h: list(g) for h, g in itertools.groupby(pack(net), lambda l: l.height)}

In [28]:
def follow_many(in_sequence, net):
    hgs = height_groups(net)
    seq = list(in_sequence)
    for h in hgs:
        for link in hgs[h]:
            seq[link.right], seq[link.left] = seq[link.left], seq[link.right]
    return seq

In [29]:
''.join(follow_many('abcdef', small_net))

'acfbed'

In [69]:
for i in range(len(small_net)+1):
    pre_net = small_net[:i]
    if i == 0:
        print('{:2}'.format(i), 
          "      ".format(small_net[i-1].left, small_net[i-1].right),
          follow_many("012345", pre_net))
    else:
        print('{:2}'.format(i), 
          "({}, {})".format(small_net[i-1].left, small_net[i-1].right),
          follow_many("012345", pre_net))

        

 0        ['0', '1', '2', '3', '4', '5']
 1 (2, 5) ['0', '1', '5', '3', '4', '2']
 2 (1, 4) ['0', '4', '5', '3', '1', '2']
 3 (0, 3) ['3', '4', '5', '0', '1', '2']
 4 (0, 3) ['0', '4', '5', '3', '1', '2']
 5 (0, 5) ['2', '4', '5', '3', '1', '0']
 6 (3, 5) ['2', '4', '5', '0', '1', '3']
 7 (0, 2) ['5', '4', '2', '0', '1', '3']
 8 (3, 4) ['5', '4', '2', '1', '0', '3']
 9 (2, 4) ['5', '4', '0', '1', '2', '3']
10 (1, 2) ['5', '0', '4', '1', '2', '3']
11 (0, 4) ['2', '0', '4', '1', '5', '3']
12 (1, 2) ['2', '4', '0', '1', '5', '3']
13 (2, 4) ['2', '4', '5', '1', '0', '3']
14 (0, 4) ['0', '4', '5', '1', '2', '3']
15 (1, 4) ['0', '2', '5', '1', '4', '3']


In [66]:
small_net[:15]

[Link(height=0, left=2, right=5),
 Link(height=1, left=1, right=4),
 Link(height=2, left=0, right=3),
 Link(height=3, left=0, right=3),
 Link(height=4, left=0, right=5),
 Link(height=5, left=3, right=5),
 Link(height=6, left=0, right=2),
 Link(height=7, left=3, right=4),
 Link(height=8, left=2, right=4),
 Link(height=9, left=1, right=2),
 Link(height=10, left=0, right=4),
 Link(height=11, left=1, right=2),
 Link(height=12, left=2, right=4),
 Link(height=13, left=0, right=4),
 Link(height=14, left=1, right=4)]

In [30]:
%%timeit
follow_many('abcdefghij', small_net)

10000 loops, best of 3: 42 µs per loop


In [37]:
''.join(follow_many(string.ascii_lowercase, net))

'doqzmbishkwunvltpcexyjgfra'

In [39]:
''.join(follow_many(string.ascii_lowercase, permnet))

'zfrasxwigvjoembqcyhplnktud'

In [42]:
''.join(follow_many(string.ascii_lowercase, rpermnet))

'doqzmbishkwunvltpcexyjgfra'

In [43]:
follow_many(string.ascii_lowercase, net) == follow_many(string.ascii_lowercase, rpermnet)

True

In [32]:
%%timeit
follow_many(string.ascii_lowercase, net)

10 loops, best of 3: 19.3 ms per loop


In [33]:
%%timeit
follow_many(string.ascii_lowercase, pnet)

100 loops, best of 3: 18.7 ms per loop
